# Compare performances

In [1]:
import os
import ast
import json
import random

import pandas as pd

# Sklearn imports
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, auc, roc_curve

from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-1-7c20b76cd595>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


### Set init variables

In [2]:
# Choose PICOS entity

picos = 'i'

## utils

In [3]:
def flatten(d):
    l = [ v for k,v in d.items() ]
    l = [item for sublist in l for item in sublist]
    l = list(map(int, l))
    return l

In [4]:
def binarize_fine(l):
    binarized_labels = []
    
    for i in l:
        if i > 1:
            i = 1
            binarized_labels.append( i )
        else:
            binarized_labels.append( i )
    
    return binarized_labels

In [5]:
def are_n_consecutive(numbers, n):
    n = 2
    if len(numbers) < n:
        return False
    for i in range(len(numbers) - n + 1):
        window = numbers[i:i+n]
        if max(window) - min(window) != n-1:
            continue
        if len(set(window)) == n:
            return True
    return False

In [6]:
def get_order():
    
    order = ['lf_Professional_Society.tsv',
    'lf_Biomedical_Occupation_or_Discipline.tsv',
    'lf_Idea_or_Concept.tsv',
    'lf_Classification.tsv',
    'lf_Biomedical_or_Dental_Material.tsv',
    'lf_Manufactured_Object.tsv',
    'lf_Functional_Concept.tsv',
    'lf_Temporal_Concept.tsv',
    'lf_Gene_or_Genome.tsv',
    'lf_Organic_Chemical.tsv',
    'lf_Biologically_Active_Substance.tsv',
    'lf_Medical_Device.tsv',
    'lf_Pharmacologic_Substance.tsv',
    'lf_Intellectual_Product.tsv',
    'lf_Diagnostic_Procedure.tsv',
    'lf_Laboratory_Procedure.tsv',
    'lf_Health_Care_Activity.tsv',
    'lf_Therapeutic_or_Preventive_Procedure.tsv',
    'lf_Finding.tsv']
    
    
    return order

## groundtruth

In [7]:
gt_p_file = '/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_bound/p/lf_Sign_or_Symptom.tsv'
gt_i_file = '/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_bound/i/lf_Therapeutic_or_Preventive_Procedure.tsv'
gt_o_file = '/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_bound/o/lf_Quantitative_Concept.tsv'

In [8]:
# load groundtruth for coarse matches

if picos == 'p':
    gt_file = gt_p_file
if picos == 'i':
    gt_file = gt_i_file
if picos == 'i':
    gt_file = gt_o_file

# open file and load into a dataframe
groundtruth_df = pd.read_csv(gt_file, sep='\t', header=0)

In [9]:
def preprocess_gt(picos_x):
    # process the coarse and fine and add the pmid:gt to a ob dictionaries
    gtcoarse = dict()
    gtfine = dict()

    for identifier, x, y in zip(groundtruth_df['pmid'], groundtruth_df[picos_x], groundtruth_df[str(picos_x)+'_f']):
        x = ast.literal_eval(x)
        y = ast.literal_eval(y)

        gtcoarse[str(identifier)] = x
        gtfine[str(identifier)] = y
    
    return gtcoarse, gtfine

In [10]:
gt_coarse, gt_fine = preprocess_gt(picos)

In [11]:
print( 'Total number of PMIDs for groundtruth - coarse: ', len(gt_coarse) )
print( 'Total number of PMIDs for groundtruth - fine: ', len(gt_fine) )

Total number of PMIDs for groundtruth - coarse:  4802
Total number of PMIDs for groundtruth - fine:  4802


In [12]:
# flatten groundtruth

gt_coarse_flattened = flatten( gt_coarse )
gt_fine_flattened = flatten( gt_fine )

# binarize the fine groundtruth labels
gt_fine_flattened = binarize_fine(gt_fine_flattened)

## order-bound

In [13]:
in_order = get_order()

In [14]:
# load order bound matches for all dictionaries

ob_dir = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_bound/{picos}'
ob_files = os.listdir(ob_dir)

In [15]:
ob_files

['lf_Medical_Device.tsv',
 'lf_Laboratory_Procedure.tsv',
 'lf_Manufactured_Object.tsv',
 'lf_Pharmacologic_Substance.tsv',
 'lf_Diagnostic_Procedure.tsv',
 'lf_Therapeutic_or_Preventive_Procedure.tsv',
 'lf_Organic_Chemical.tsv',
 'lf_Biologically_Active_Substance.tsv',
 'lf_Idea_or_Concept.tsv',
 'lf_Professional_Society.tsv',
 'lf_Temporal_Concept.tsv',
 'lf_Finding.tsv',
 'lf_Biomedical_or_Dental_Material.tsv',
 'lf_Functional_Concept.tsv',
 'lf_Biomedical_Occupation_or_Discipline.tsv',
 'lf_Gene_or_Genome.tsv',
 'lf_Health_Care_Activity.tsv',
 'lf_Classification.tsv',
 'lf_Intellectual_Product.tsv']

In [16]:
ob_matches = dict()

for i in tqdm(ob_files):
    print(i)
    file_path = f'{ob_dir}/{i}'
    
    filename = str(i).replace('lf_', '')
    filename = str(filename).replace('.tsv', '')
    
    # open file and load into a dataframe
    data_df = pd.read_csv(file_path, sep='\t', header=0)
    
    # process the labels and add the pmid:labels to a ob dictionary
    labs_all = []
    for x in data_df['labels']:
        x = ast.literal_eval(x)
        labs_all.append( x )
    ob_matches[filename] = dict( zip( data_df['pmid'],  pd.Series( labs_all )) )

  0%|          | 0/19 [00:00<?, ?it/s]

lf_Medical_Device.tsv


  5%|▌         | 1/19 [00:03<01:07,  3.75s/it]

lf_Laboratory_Procedure.tsv


 11%|█         | 2/19 [00:07<01:05,  3.83s/it]

...

 89%|████████▉ | 17/19 [01:04<00:07,  3.86s/it]

lf_Classification.tsv


 95%|█████████▍| 18/19 [01:08<00:03,  3.78s/it]

lf_Intellectual_Product.tsv


100%|██████████| 19/19 [01:12<00:00,  3.80s/it]


In [17]:
random_key = random.sample(ob_matches.keys(), 1)[0]

In [18]:
print( 'Total number of PMIDs for orderbound files: ', len(ob_matches[random_key]) )

Total number of PMIDs for orderbound files:  4802


In [19]:
assert len(gt_fine) == len( ob_matches[random_key] ) 

In [20]:
len(gt_fine)

4802

In [22]:
# calculate performance metrics for order-bound files

for k, v in ob_matches.items():
    
    v_flattened = flatten( v )
    
    # Classification report
    cr_order_bound_coarse = classification_report( gt_coarse_flattened, v_flattened, digits=4, output_dict=True)
    print(f'Confusion matrix for coarse-grained ground truth and {k} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_coarse['macro avg']['recall'], cr_order_bound_coarse['macro avg']['f1-score'], cr_order_bound_coarse['0']['precision'], cr_order_bound_coarse['0']['recall'], cr_order_bound_coarse['0']['f1-score'], cr_order_bound_coarse['1']['precision'], cr_order_bound_coarse['1']['recall'], cr_order_bound_coarse['1']['f1-score'])
    print( report )
    

    cr_order_bound_fine = classification_report( gt_fine_flattened, v_flattened, digits=4, output_dict=True)
    print(f'\n\nConfusion matrix for fine-grained ground truth and {k} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_fine['macro avg']['recall'], cr_order_bound_fine['macro avg']['f1-score'], cr_order_bound_fine['0']['precision'], cr_order_bound_fine['0']['recall'], cr_order_bound_fine['0']['f1-score'], cr_order_bound_fine['1']['precision'], cr_order_bound_fine['1']['recall'], cr_order_bound_fine['1']['f1-score'])
    print( report )
    
    print( '--------------------------------------------------------------------------' )

Confusion matrix for coarse-grained ground truth and Medical_Device matches
0.5084, 0.4959, 0.9048, 0.9917, 0.9463, 0.2454, 0.0251, 0.0455


Confusion matrix for fine-grained ground truth and Medical_Device matches
0.5108, 0.5067, 0.9321, 0.9916, 0.9610, 0.2103, 0.0300, 0.0525
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and Laboratory_Procedure matches
0.5155, 0.5110, 0.9061, 0.9841, 0.9435, 0.2400, 0.0469, 0.0785


Confusion matrix for fine-grained ground truth and Laboratory_Procedure matches
0.5207, 0.5241, 0.9335, 0.9840, 0.9580, 0.2105, 0.0574, 0.0903
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and Manufactured_Object matches
0.5038, 0.4921, 0.9040, 0.9821, 0.9414, 0.1320, 0.0254, 0.0427


Confusion matrix for fine-grained ground truth and Manufactured_Object matches
0.5061, 0.5018, 0.9315, 0.9822, 0.9562, 0.1117, 0.0300, 0

## order-free

In [23]:
# load order free matches with all dictionaries

picos = 'I' # ['P', 'I', 'O', 'S']
match_level = 'doc' # ['doc', 'sent', 'win_5', 'para']

In [24]:
order_free_dir = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_free/{match_level}/{picos}'
order_free_files = os.listdir(order_free_dir)
print('Files: ', order_free_files)

Files:  ['Biomedical_or_Dental_Material.json', '.nfs0000000011200a4200000003', 'Classification.json', 'Intellectual_Product.json', 'Biologically_Active_Substance.json', 'Diagnostic_Procedure.json', 'Gene_or_Genome.json', 'Finding.json', 'Functional_Concept.json', 'Medical_Device.json', 'Organic_Chemical.json', 'Laboratory__Procedure.json', 'Manufactured_Object.json', 'train_ebm_intervention.json', 'Professional_Society.json', 'Pharmacologic_Substance.json', 'Therapeutic_or_Preventive_Procedure.json', 'train_ebm_intervention_syn.json', 'Biomedical_Occupation_or_Discipline.json', 'Health_Care_Activity.json', 'Idea_or_Concept.json', 'Temporal_Concept.json']


In [25]:
order_free_files.remove('.nfs0000000011200a4200000003')
#order_free_files.remove('Finding.json')

In [26]:
orf_loaded_files = dict()

for i in tqdm(order_free_files):
    
    filpath = f'{order_free_dir}/{i}'
    print('Loading file...', i)
    with open( filpath, 'r' ) as rf:
        orf_i = json.load(rf)
        orf_loaded_files[i] = orf_i

  0%|          | 0/21 [00:00<?, ?it/s]

Loading file... Biomedical_or_Dental_Material.json


  5%|▍         | 1/21 [00:21<07:16, 21.85s/it]

Loading file... Classification.json


 10%|▉         | 2/21 [00:23<03:06,  9.81s/it]

...

 90%|█████████ | 19/21 [12:26<01:37, 48.75s/it]

Loading file... Idea_or_Concept.json


 95%|█████████▌| 20/21 [12:26<00:34, 34.90s/it]

Loading file... Temporal_Concept.json


100%|██████████| 21/21 [12:27<00:00, 35.62s/it]


In [27]:
# set variables

set_partial_matches = True
set_actual_orf_matches = True

In [28]:
def get_orf(v, par, actual_orf):
    
    orfs = dict()
    
    for k_i, v_i in v.items():
        # print( k_i ) # example: name_15_3, name_9_9, name_12_8
        
        if 'Inters. (full)' in v_i and len(v_i['Inters. (full)']) > 0:
            
            full_inters = v_i['Inters. (full)']
            # full_inters.keys() = PMIDs
            # full_inters.values() = offsets, tokens

            for pmid, matches in full_inters.items():
                if len(matches['char offs.']) > 1:
                    if pmid not in orfs:
                        if actual_orf == True:
                            #if sorted( matches['word offs.'] ) != matches['word offs.']:
                            if are_n_consecutive(matches['word offs.'], 2) == False:
                                orfs[pmid] = [ ]
                                orfs[pmid].extend( matches['word offs.'] )      
                        elif actual_orf == False: 
                            orfs[pmid] = [ ]
                            orfs[pmid].extend( matches['word offs.'] )
                    else:
                        if actual_orf == True:
                            #if sorted( matches['word offs.'] ) != matches['word offs.']:
                            if are_n_consecutive(matches['word offs.'], 2) == False:
                                orfs[pmid].extend( matches['word offs.'] )
                        elif actual_orf == False:
                            orfs[pmid].extend( matches['word offs.'] )


        if 'Inters. (partial)' in v_i and len(v_i['Inters. (partial)']) > 0 and (par=='both' or par==True):
            
            par_inters = v_i['Inters. (partial)']
            
            for pmid, matches in par_inters.items():
                if len(matches['char offs.']) > 1:
                    if pmid not in orfs:
                        if actual_orf == True:
                            #if sorted( matches['word offs.'] ) != matches['word offs.']:
                            if are_n_consecutive(matches['word offs.'], 2) == False:
                                orfs[pmid] = [ ]
                                orfs[pmid].extend( matches['word offs.'] )   
                        elif actual_orf == False:
                            orfs[pmid] = [ ]
                            orfs[pmid].extend( matches['word offs.'] ) 
                    else:
                        if actual_orf == True:
                            #if sorted( matches['word offs.'] ) != matches['word offs.']:
                            if are_n_consecutive(matches['word offs.'], 2) == False:
                                orfs[pmid].extend( matches['word offs.'] )
                        elif actual_orf == False:
                            orfs[pmid].extend( matches['word offs.'] )
    
    return orfs

In [29]:
orfs_dict = dict()

for k,v in orf_loaded_files.items():
    
    new_k = str(k).replace('.json', '')

    orf_fetched = get_orf(v, par = set_partial_matches, actual_orf = set_actual_orf_matches)
    orfs_dict[new_k] = orf_fetched

In [30]:
# Order the loaded ORF matches in ascending order depending on the number of matches found

order_offset_dict = dict()

for k,v in orfs_dict.items():
    
    total_offsets = []
    for k_i, v_i in v.items():
        total_offsets.extend( v_i )
    
    order_offset_dict[k] = len(total_offsets)
    
    print( f'Offsets in {k}: ', len(total_offsets) )

Offsets in Biomedical_or_Dental_Material:  269764
Offsets in Classification:  221903
Offsets in Intellectual_Product:  6384570
Offsets in Biologically_Active_Substance:  1747134
Offsets in Diagnostic_Procedure:  8650791
Offsets in Gene_or_Genome:  707175
Offsets in Finding:  34806379
Offsets in Functional_Concept:  438148
Offsets in Medical_Device:  2408193
Offsets in Organic_Chemical:  1226365
Offsets in Laboratory__Procedure:  9350649
Offsets in Manufactured_Object:  351652
Offsets in train_ebm_intervention:  20569
Offsets in Professional_Society:  1014
Offsets in Pharmacologic_Substance:  3235747
Offsets in Therapeutic_or_Preventive_Procedure:  80134307
Offsets in train_ebm_intervention_syn:  74471
Offsets in Biomedical_Occupation_or_Discipline:  29481
Offsets in Health_Care_Activity:  13611371
Offsets in Idea_or_Concept:  168142
Offsets in Temporal_Concept:  656504


In [31]:
# sort dictionary
order_offset_sorteddict = dict(sorted(order_offset_dict.items(), key=lambda x: x[1]))

In [32]:
print( 'Total number of dictionaries loaded: ', len(order_offset_sorteddict) )

Total number of dictionaries loaded:  21


In [33]:
# temporary - to remove
del order_offset_sorteddict['train_ebm_intervention_syn']
del order_offset_sorteddict['train_ebm_intervention']

orfs_dict["Laboratory_Procedure"] = orfs_dict.pop("Laboratory__Procedure")
order_offset_sorteddict["Laboratory_Procedure"] = order_offset_sorteddict.pop("Laboratory__Procedure")

In [34]:
assert len( order_offset_sorteddict.keys() ) == len( ob_matches.keys() )
assert sorted( order_offset_sorteddict.keys() ) == sorted( ob_matches.keys() )

In [35]:
list( order_offset_sorteddict.keys() )

['Professional_Society',
 'Biomedical_Occupation_or_Discipline',
 'Idea_or_Concept',
 'Classification',
 'Biomedical_or_Dental_Material',
 'Manufactured_Object',
 'Functional_Concept',
 'Temporal_Concept',
 'Gene_or_Genome',
 'Organic_Chemical',
 'Biologically_Active_Substance',
 'Medical_Device',
 'Pharmacologic_Substance',
 'Intellectual_Product',
 'Diagnostic_Procedure',
 'Health_Care_Activity',
 'Finding',
 'Therapeutic_or_Preventive_Procedure',
 'Laboratory_Procedure']

In [36]:
def merge_preds_ordered( ob_preds, orf_offsets ):
    
    merged_predictions = dict()
    
    for k, v in ob_preds.items(): # ob preds are the old preds
        
        k = str(k)

        if k in orf_offsets:
            matching_offsets = orf_offsets[k]
            old_preds = list( map( int, v ))
            new_preds = list( map( int, v ))
                             
            # add new offsets to the new_preds
            for indice in matching_offsets:
                new_preds[indice] = 1

            merged_predictions[ k ] = new_preds
        else:
            merged_predictions[ k ] = list( map( int, v ) )
    
    return merged_predictions

In [37]:
# full matches - Order free matches
for count, i in enumerate(in_order):
    
    to_fetch = str(i).replace('lf_', '').replace('.tsv', '')
    
    base_preds = ob_matches[to_fetch]
    base_preds_flattened =  flatten( base_preds )
    
    updated_of_preds = merge_preds_ordered( base_preds, orfs_dict[to_fetch] )
    updated_preds_flattened =  flatten( updated_of_preds )
    
    # Classification report
    cr_order_bound_coarse = classification_report( gt_coarse_flattened, updated_preds_flattened, digits=4, output_dict=True)
    print(f'Confusion matrix for coarse-grained ground truth and {i} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_coarse['macro avg']['recall'], cr_order_bound_coarse['macro avg']['f1-score'], cr_order_bound_coarse['0']['precision'], cr_order_bound_coarse['0']['recall'], cr_order_bound_coarse['0']['f1-score'], cr_order_bound_coarse['1']['precision'], cr_order_bound_coarse['1']['recall'], cr_order_bound_coarse['1']['f1-score'])
    print( report )

    cr_order_bound_fine = classification_report( gt_fine_flattened, updated_preds_flattened, digits=4, output_dict=True)
    print(f'\n\nConfusion matrix for fine-grained ground truth and {i} matches')
    report = '{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}'.format(cr_order_bound_fine['macro avg']['recall'], cr_order_bound_fine['macro avg']['f1-score'], cr_order_bound_fine['0']['precision'], cr_order_bound_fine['0']['recall'], cr_order_bound_fine['0']['f1-score'], cr_order_bound_fine['1']['precision'], cr_order_bound_fine['1']['recall'], cr_order_bound_fine['1']['f1-score'])
    print( report )
    
    print( '--------------------------------------------------------------------------' )

Confusion matrix for coarse-grained ground truth and lf_Professional_Society.tsv matches
0.5000, 0.4751, 0.9033, 0.9994, 0.9489, 0.0948, 0.0006, 0.0012


Confusion matrix for fine-grained ground truth and lf_Professional_Society.tsv matches
0.5000, 0.4825, 0.9307, 0.9994, 0.9639, 0.0701, 0.0006, 0.0012
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and lf_Biomedical_Occupation_or_Discipline.tsv matches
0.5035, 0.4872, 0.9040, 0.9910, 0.9455, 0.1592, 0.0160, 0.0290


Confusion matrix for fine-grained ground truth and lf_Biomedical_Occupation_or_Discipline.tsv matches
0.5054, 0.4975, 0.9314, 0.9911, 0.9603, 0.1411, 0.0197, 0.0346
--------------------------------------------------------------------------
Confusion matrix for coarse-grained ground truth and lf_Idea_or_Concept.tsv matches
0.4847, 0.4849, 0.9004, 0.8971, 0.8987, 0.0700, 0.0724, 0.0711


Confusion matrix for fine-grained ground truth and lf_Idea_or_C